### Elasticsearch 
# Almacén de documentos y Motor de búsqueda 

<p style="font-size: large; margin-top: 100px;">César de Pablo Sánchez</p>
<p style="font-size: large">@zdepablo</p>

## Características principales

 * Orientado a documentos - almacena documentos (objetos complejos vs tablas) 
 * Documento: objeto principal que se serializa como JSON y se almacena con un ID único
 * Indexación: ES indexa **todos** los campos por defecto
 * Tipos (*types*) : Los documentos pueden tener tipos: 
    * Ejemplo: entradas, usuarios, comentarios 

## Características principales
 
 * Cada tipo tiene su propio esquema definido mediante un *mapping*
 * Los documentos son **inmutables**: si se cambia o reemplaza el documento se tiene que indexar otra vez. 
    * La version se incrementa 

## REST API

  * La mayor parte de la funcionalidad de ES está disponible mediante una API REST (HTTP)
  * Diseño intuitivo - permite ir descubriendo la funcionalidad poco a poco
  
  
  * Podemos usar el navegador o CURL 
  * ES 2.0 proporciona una interfaz de consulta: Sense


## REST API

<pre>
curl -X&lt;VERB&gt;

'&lt;PROTOCOL&gt;://&lt;HOST&gt;:&lt;PORT&gt;/&lt;PATH&gt;?&lt;QUERY_STRING&gt;' 

-d '&lt;BODY&gt;'
</pre>


<pre>
curl -XGET 

'http://localhost:9200/_count?pretty' 

-d '{
  "query": { 
      "match_all": {}
  }
}
'
</pre>


* VERB-The appropriate HTTP method or verb: GET, POST, PUT, HEAD, or DELETE.
* PROTOCOL-Either http or https (if you have an https proxy in front of Elasticsearch.)
* HOST-The hostname of any node in your Elasticsearch cluster, or localhost for a node on your local machine.
* PORT-The port running the Elasticsearch HTTP service, which defaults to 9200.
* PATH-API Endpoint (for example _count will return the number of documents in the cluster). Path may contain multiple components, such as _cluster/stats or _nodes/stats/jvm
* QUERY_STRING-Any optional query-string parameters (for example ?pretty will pretty-print the JSON response to make it easier to read.)
* BODY-A JSON-encoded request body (if the request needs one.)

## REST API - Verbos 

Permiten realizar las principales operaciones CRUD ( CREATE , READ, UPDATE, DELETE) 

Verbos HTTP: 
   - **GET**  - Lee un recurso 
   - **POST** - Modifica un recurso
   - **PUT**  - Crea un recurso - en los casos en los que tiene sentido 
   - **HEAD** 
   - **DELETE** - Borra un recurso 

## REST API - Recursos 
Recursos: se identifican mediante el <PATH>
  - Documentos, Índices, Mappings
  - Cluster, nodos 

## Creando un documento (indexación)

<pre>
PUT /&lt;INDEX&gt;/&lt;TYPE&gt;/&lt;ID&gt; 

&lt;JSON_DOC&gt;
</pre>


  - Crea el documento en ES 
  - Si el índice no existe lo crea automáticamente 
  - Si el tipo no existe lo crea automáticamente y genera un mapping "adecuado" al documento.


## Creando un documento

In [1]:
import requests

In [2]:
employee = """
{
    "first_name" : "John",
    "last_name" :  "Smith",
    "age" :        25,
    "about" :      "I love to go rock climbing",
    "interests": [ "sports", "music" ]
}
"""

r = requests.put('http://localhost:9200/megacorp/employee/1?pretty', 
                 data = employee)

## Creando un documento

In [3]:
print r.text

{
  "_index" : "megacorp",
  "_type" : "employee",
  "_id" : "1",
  "_version" : 10,
  "created" : false
}



[TODO] Explicar el significado de cada parte de la url
[TODO] Explicar los diferentes tipos de campos: texto, numéricos, listas, datetimes

ES creates if not exist:

 * an **index** (*megacorp*)  
 * a **document type** (*employee*)

### Metadatos del documento

Cada documento tiene metadatos asociados:

 - **_index**: indice en el que se almacena
 - **_type**:  clase de objeto que representa
 - **_id**:    identificador único del documento 
   - Es siempre una cadena (UUIDs), en concreto tiene 20 caracteres, es URL-safe, codificada en Base64 
   - Podemos crear nuestros ids o dejar a ES que lo haga por nosotros
  
Los tres ( _index, _type y _id) identifican univocamente al documento:  

#### Otros metadatos

 - **_version** - version del documento
 - **_timestamp** - tiempo de indexacion - (*deshabilitado*)
 - **_ttl** - tiempo de vida (*deshabilitado*)
  

### Creando más documentos de ejemplo

In [4]:
employee2 = """
{
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}
"""

employee3 = """
{
    "first_name" :  "Douglas",
    "last_name" :   "Fir",
    "age" :         35,
    "about":        "I like to build cabinets",
    "interests":  [ "forestry" ]
}
"""

r = requests.put('http://localhost:9200/megacorp/employee/2', data = employee2)
r = requests.put('http://localhost:9200/megacorp/employee/3', data = employee3)


### Obteniendo un documento por id

In [5]:
r = requests.get('http://localhost:9200/megacorp/employee/2?pretty')
print r.text

{
  "_index" : "megacorp",
  "_type" : "employee",
  "_id" : "2",
  "_version" : 2,
  "found" : true,
  "_source":
{
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}

}



### Obteniendo un documento que no existe

In [6]:
r = requests.get('http://localhost:9200/megacorp/employee/200?pretty')
print 'Status: ', r.status_code, '\n'
print r.text

Status:  404 

{
  "_index" : "megacorp",
  "_type" : "employee",
  "_id" : "200",
  "found" : false
}



### Recuperando solo una parte del documento

aka. SELECT fields 

In [7]:
r = requests.get('http://localhost:9200/megacorp/employee/2?_source=first_name,last_name&pretty')
print r.text

{
  "_index" : "megacorp",
  "_type" : "employee",
  "_id" : "2",
  "_version" : 2,
  "found" : true,
  "_source":{"first_name":"Jane","last_name":"Smith"}
}



### Recuperando el contenido original - sin metadatos

In [8]:
r = requests.get('http://localhost:9200/megacorp/employee/2/_source')
print r.text


{
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}



## Borrando un documento

- No elimina de forma inmediata el documento del disco
- Marca el documento como borrado 
- Limpia los documentos marcados como borrados en segundo plano 
- Aprovecha para hacer tareas de mantenimiento de los índices. 

## Actualizando un documento 
- Se borra el documento antiguo 
- Se crea un nuevo documento con el mismo _id pero nueva _version 
- En las nuevas versiones - se hace de forma "atómica"

### Actualizar parte de un documento 
<pre>POST /_update </pre> 
### Otras operaciones sobre los documentos 
   - permite añadir campos
   - modificar campos
   - mezclar un documento - por ejemplo podemos añadir valores a un campo de tipo coleccion


- Indexar un documento - PUT 
- Ontener un documento - GET
- Borrar un documento  - DELETE 
- Comprobar si existe  - HEAD  
- Actualizar un documento existente - basta con volver a hacer PUT de todo el documento

### Otras operaciones sobre los documentos 

- Actualizar parte de un documento -POST /_update 
- Crear y asignarle un id único     - POST
- Crear si no existe                - POST /_create
- Obtener multiples documentos en una sola petición - GET /mget or /&lt;index&gt;/&lt;type&gt;/_mget
- Ejecutar acciones en bloque(bulk)  -bulk API

### Almacén de datos distribuido

- ES usa **control optimista de concurrencia**
- asume que los conflictos van a ser infrecuentes
- Generalmente, no se usa como base de datos primaria. Si se hace, es la aplicación la que tiene que resolver los problemas.

- Documentos se replican en varios nodos del cluster.
- Indices estan particionados (shards) y distribuidos N veces


### Problemas con el control optimista de la concurrencia

<img src="https://www.elastic.co/guide/en/elasticsearch/guide/master/images/elas_0301.png" height="300" width="300" >


## Buscando documentos

<pre>
GET /&lt;INDEX&gt;/&lt;TYPE&gt;/_search 
</pre>


Buscando todos los documentos de un tipo: 

In [9]:
r = requests.get('http://localhost:9200/megacorp/employee/_search?pretty')

## Buscando documentos

In [10]:
print r.text

{
  "took" : 12,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 3,
    "max_score" : 1.0,
    "hits" : [ {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "2",
      "_score" : 1.0,
      "_source":
{
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}

    }, {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "1",
      "_score" : 1.0,
      "_source":
{
    "first_name" : "John",
    "last_name" :  "Smith",
    "age" :        25,
    "about" :      "I love to go rock climbing",
    "interests": [ "sports", "music" ]
}

    }, {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "3",
      "_score" : 1.0,
      "_source":
{
    "first_name" :  "Douglas",
    "last_name" :   "Fir",
    "age" :         35,
    "about":        "I lik

## Buscando documentos

 - **_hits**  - Lista de documentos que cumplen los criterios de búsqueda. SE compone de los N mejores documentos 
 - **_score** - Relevancia del documento para la consulta. La lista de hits generalmente se ordena de mayor a menor relevancia  
 - Otros metadatos del proceso de búsqueda, como la explicación de la relevancia. 

[TODO] Explain the URL: 
  - *_search* 
  - parameter *q* 
  - field queries: *last_name:*   

## Buscando documentos

Busca los documentos relevantes para la necesidad de información: "El apellido es Smith"  


In [ ]:
r = requests.get('http://localhost:9200/megacorp/employee/_search?q=last_name:Smith&pretty')

In [11]:
print r.text    

{
  "took" : 25,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 1.0,
    "hits" : [ {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "2",
      "_score" : 1.0,
      "_source":
{
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}

    }, {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "1",
      "_score" : 1.0,
      "_source":
{
    "first_name" : "John",
    "last_name" :  "Smith",
    "age" :        25,
    "about" :      "I love to go rock climbing",
    "interests": [ "sports", "music" ]
}

    } ]
  }
}



## Buscando documentos
Buscando en un campo de texto

In [ ]:
r = requests.get('http://localhost:9200/megacorp/employee/_search?q=about:rock&pretty')

In [12]:
print r.text 

{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 0.375,
    "hits" : [ {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "2",
      "_score" : 0.375,
      "_source":
{
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}

    }, {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "1",
      "_score" : 0.375,
      "_source":
{
    "first_name" : "John",
    "last_name" :  "Smith",
    "age" :        25,
    "about" :      "I love to go rock climbing",
    "interests": [ "sports", "music" ]
}

    } ]
  }
}



### Buscando documentos con Query DSL

- Query DSL - Lenguaje específico de dominio para búsqueda y analítica

We can rewrite the simple query as a __match__ query 

In [ ]:
payload = """
{
    "query" : {
        "match" : {
            "last_name" : "Smith"
        }
    }
}
"""

r = requests.get('http://localhost:9200/megacorp/employee/_search?pretty', data = payload)

In [13]:
print r.text    

{
  "took" : 10,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 1.0,
    "hits" : [ {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "2",
      "_score" : 1.0,
      "_source":
{
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}

    }, {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "1",
      "_score" : 1.0,
      "_source":
{
    "first_name" : "John",
    "last_name" :  "Smith",
    "age" :        25,
    "about" :      "I love to go rock climbing",
    "interests": [ "sports", "music" ]
}

    } ]
  }
}



### Query DSL - Aplicando filtros

Podemos combinar consultas y filtros para trabajar con info estructurada y no estructurada

Selecciona solo empleados con **edad>30** y **apellido = "Smith"**

In [ ]:
payload = """
{
    "query" : {
        "filtered" : {
            "filter" : {
                "range" : {
                    "age" : { "gt" : 30 } 
                }
            },
            "query" : {
                "match" : {
                    "last_name" : "smith" 
                }
            }
        }
    }
}
"""

r = requests.get('http://localhost:9200/megacorp/employee/_search?pretty', data = payload)

In [14]:
print r.text    

{
  "took" : 20,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 1.0,
    "hits" : [ {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "2",
      "_score" : 1.0,
      "_source":
{
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}

    } ]
  }
}



### Query DSL - Búsqueda sobre texto completo
- Gran variedad de consultas complejas 
- Relevancia: cada documento tiene una relevancia calculada para la consulta concreta 
   

In [ ]:
payload = """
{
    "query" : {
        "match" : {
            "about" : "rock climbing"
        }
    }
}
"""

r = requests.get('http://localhost:9200/megacorp/employee/_search?pretty', data = payload)

In [15]:
print r.text    

{
  "took" : 21,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 2,
    "max_score" : 0.6468432,
    "hits" : [ {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "1",
      "_score" : 0.6468432,
      "_source":
{
    "first_name" : "John",
    "last_name" :  "Smith",
    "age" :        25,
    "about" :      "I love to go rock climbing",
    "interests": [ "sports", "music" ]
}

    }, {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "2",
      "_score" : 0.108701006,
      "_source":
{
    "first_name" :  "Jane",
    "last_name" :   "Smith",
    "age" :         32,
    "about" :       "I like to collect rock albums",
    "interests":  [ "music" ]
}

    } ]
  }
}



### Query DSL - Búsqueda de frases

-  **match: rock climbing** - contienen *rock* o *climbing*, obtienen mejor score si ambas aparecen
-  **match_phrase: "rock climbing"** - contiene la frase *"rock climbing"*, ambos terminos tienen que aparecer, contiguos y en orden. 


In [ ]:
payload = """
{
    "query" : {
        "match_phrase" : {
            "about" : "rock climbing"
        }
    }
}
"""

r = requests.get('http://localhost:9200/megacorp/employee/_search?pretty', data = payload)

In [16]:
print r.text    

{
  "took" : 22,
  "timed_out" : false,
  "_shards" : {
    "total" : 1,
    "successful" : 1,
    "failed" : 0
  },
  "hits" : {
    "total" : 1,
    "max_score" : 0.9020494,
    "hits" : [ {
      "_index" : "megacorp",
      "_type" : "employee",
      "_id" : "1",
      "_score" : 0.9020494,
      "_source":
{
    "first_name" : "John",
    "last_name" :  "Smith",
    "age" :        25,
    "about" :      "I love to go rock climbing",
    "interests": [ "sports", "music" ]
}

    } ]
  }
}



## Paginado de resultados